In [ ]:
# Install package not available in Colab by default
!pip install boruta

In [ ]:
# Data load and processing
from google.colab import files
import io
import numpy as np
import pandas as pd

# Models-related
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, OrthogonalMatchingPursuit
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import normalize, StandardScaler
from sklearn import metrics

from boruta import BorutaPy

# Visualisations
import matplotlib.pyplot as plt
import seaborn as sns

# Numbers display set-up
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.float_format", lambda x: "%.7f" % x)

# 1) Data Preparation

In [ ]:
uploaded = files.upload()
df = pd.read_csv(io.BytesIO(uploaded["hr-data.csv"]))

In [ ]:
df.describe(include="all")

In [ ]:
df.head()

In [ ]:
# replace NaNs with string so we create corresponding category later on
df.replace(np.nan, "nan", regex=True, inplace=True)
# parse employee's current rating
df["current_rating"] = df.current_rating.str[0].astype(int)
# drop features with explanatory power
df.drop(columns=["legal_full_name", "employee_id", "manager_id"], inplace=True)

In [ ]:
# Create categorical variables because our linear regression based methods accept only numerical values as an input
worker_data = pd.get_dummies(df, prefix_sep=" -> ")
print(f"Our dataset has {worker_data.shape[0]} rows and {worker_data.shape[1]} columns.")

In [ ]:
worker_data.head()

In [ ]:
# Example of one hot encoding
df.country.head()

In [ ]:
worker_data[[c for c in worker_data.columns if c.startswith("country")]].head()

# 2) Linear Regression

There are a few assumptions associated with a linear regression model:

* Linearity: The relationship between X and Y is linear.
* Homoscedasticity: The variance of residual is the same for any value of X.
* No multicollinearity: There is no high correlations among two or more independent variables.
* Independence: Observations are independent of each other.
* Normality: For any fixed value of X, Y is normally distributed.

In [ ]:
# avoid adding additional multicollinearity by dropping one category per each feature
worker_data = pd.get_dummies(df, prefix_sep=" -> ", drop_first=True)
print(f"Our dataset has {worker_data.shape[0]} rows and {worker_data.shape[1]} columns.")

In [ ]:
X_data = worker_data.drop("total_compensation", axis=1)
y_data = worker_data["total_compensation"]

X_train, X_test, y_train, y_test = train_test_split(
    X_data, y_data, test_size=0.20, random_state=12
)
column_names = X_train.columns

In [ ]:
# Standard scaler uses this formula: z = (x - u) / s
# which results in 0 mean 1 std for all features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(6, 5))
ax1.set_title("Before feature scaling")
sns.kdeplot(X_train["age"], ax=ax1)
sns.kdeplot(X_train["length_of_service"], ax=ax1)
ax2.set_title("After feature scaling")
sns.kdeplot(X_train_scaled[:, 7], ax=ax2)
sns.kdeplot(X_train_scaled[:, 6], ax=ax2)
plt.show()

In [ ]:
# Create a linear regression model
model = LinearRegression()
# Train the model
model.fit(X_train_scaled, y_train)
# Make predictions on the test set
y_pred = model.predict(X_test_scaled)
residuals = y_test - y_pred
# Evaluate the model
rmse = metrics.mean_squared_error(y_test, y_pred, squared=False)
r2 = metrics.r2_score(y_test, y_pred)
mape = metrics.mean_absolute_percentage_error(y_test, y_pred)

print("Mean Absolute Percentage Error (MAPE):", round(mape * 100, 2), "%")
print("Root Mean Squared Error:", rmse)
print("R-squared:", r2)

In [ ]:
# We broke multicollinearity assumption by using all features
f = plt.figure(figsize=(19, 15))
plt.matshow(X_train.corr(), fignum=f.number)
cb = plt.colorbar()
cb.ax.tick_params(labelsize=14)
plt.title('Correlation Matrix', fontsize=16);
# NOTE: White line represents columns which contain only zeros

# 3) Orthogonal Matching Pursuit

In [ ]:
# OMP model can handle multicollinearity
worker_data = pd.get_dummies(df, prefix_sep=" -> ", drop_first=False)
print(f"Our dataset has {worker_data.shape[0]} rows and {worker_data.shape[1]} columns.")

In [ ]:
X_data = worker_data.drop("total_compensation", axis=1)
y_data = worker_data["total_compensation"]

X_train, X_test, y_train, y_test = train_test_split(
    X_data, y_data, test_size=0.20, random_state=12
)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Create a OMP model
n_coefs = 20
model = OrthogonalMatchingPursuit(n_nonzero_coefs=n_coefs)
# Train the model
model.fit(X_train_scaled, y_train)
# Make predictions on the test set
y_pred = model.predict(X_test_scaled)
residuals = y_test - y_pred
# Evaluate the model
rmse = metrics.mean_squared_error(y_test, y_pred, squared=False)
r2 = metrics.r2_score(y_test, y_pred)
mape = metrics.mean_absolute_percentage_error(y_test, y_pred)

print("Mean Absolute Percentage Error (MAPE):", round(mape * 100, 2), "%")
print("Root Mean Squared Error:", rmse)
print("R-squared:", r2)

In [ ]:
# Visualise the features that have been selected and their respective scores
feature_scores = pd.Series(model.coef_, index=X_data.columns).sort_values(ascending=False, key=lambda x: abs(x))
selected_features = feature_scores[:n_coefs]

f, ax = plt.subplots(figsize=(10, 5))
shades = 31
palette = sns.color_palette("coolwarm", shades)
minmax = np.max([-np.min(selected_features), np.max(selected_features)])
bins = np.linspace(-minmax, minmax, num=shades)
palette_indices = np.digitize(selected_features, bins) - 1
colors = [palette[idx] for idx in palette_indices]

ax = sns.barplot(x=selected_features, y=selected_features.index, palette=colors, dodge=False)
ax.set_yticklabels(selected_features.index)
ax.set_xlabel("Feature impact on prediction")
plt.show()

## Exercises

### A. Implement Orthogonal Matching Pursuit: Write code to implement the Orthogonal Matching Pursuit algorithm from scratch. Given a dictionary matrix, a target signal, and a desired sparsity level, apply OMP to approximate the target signal using a sparse representation.


In [ ]:
rng = np.random.default_rng(12345)
n_samples = 10
age = rng.normal(45, 12, size=n_samples)
length_of_service = rng.normal(2, 1, size=n_samples)
current_rating = rng.integers(1, 5, endpoint=True, size=n_samples)
is_manager = rng.binomial(1, 0.2, size=n_samples)

# total compensation rounded to nearest thousand dollars
total_compensation = np.array([194, 120, 120, 115, 131, 143, 128, 165, 129, 132])

In [ ]:
input_data = pd.DataFrame(np.column_stack([age, length_of_service, current_rating, is_manager]), columns=["age", "length_of_service", "current_rating", "is_manager"])

In [ ]:
input_data

In [ ]:
total_compensation

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(input_data)

TASK: Estimate coefficients for each vector in dataset (dictionary) such that target (signal) can be approximated by their linear combination. Moreover, try to minimize number of used features (atoms).

In [ ]:
def omp(X, y, sparsity):
  ...
  # TODO


In [ ]:
# We have 4 features, let's examine their contribution for our target variable.
# The contribution is defined as scalar product of each feature with the target.

# e.g. contribution of 1st feature ("age") in the target
# element-wise multiplication and then summation
age_mul = X_train[:, 0] * total_compensation
age_contr = np.sum(age_mul)
print("Contribution of 'age' feature is:", age_contr)

In [ ]:
# We will continue with other features and their corresponding effect in the target
len_of_service_contr = np.sum(X_train[:, 1] * total_compensation)
print("Contribution of 'length_of_service' feature is:", len_of_service_contr)

current_rating_contr = np.sum(X_train[:, 2] * total_compensation)
print("Contribution of 'current_rating' feature is:", current_rating_contr)

is_manager_contr = np.sum(X_train[:, 3] * total_compensation)
print("Contribution of 'is_manager' feature is:", is_manager_contr)

In [ ]:
# Let's look at the absolute maximum contribution
contributions = [age_contr, len_of_service_contr, current_rating_contr, is_manager_contr]
abs_contributions = np.abs(contributions)

In [ ]:
# Let's get the index (feature name) which belongs to the maximum value
idx = np.argmax(abs_contributions)
print(f"'{input_data.columns[idx]}' contributes the most to the target variable.")

In [ ]:
# The code above can be rewritten using numpy matrix operations
contrs = X_train.T @ total_compensation
abs_contrs = np.abs(contrs)
idx = np.argmax(abs_contrs)
print(f"'{input_data.columns[idx]}' contributes the most to the target variable.")

In [ ]:
# Our new dataset (basis) consists of one feature ("length_of_service"). Now,
# we can estimate the coefficient for this feature. We will leverage numpy implementation
# of calculating pseudo inverse of matrix (A^+ = (A^T * A)^-1 * A^T) when finding
# the matrix that solves the least-squares problem A * x = b. This equation can
# be rewritten as x = A^-1 * b.
inverse_matrix = np.linalg.pinv(X_train[:, [idx]])
coefficient = inverse_matrix @ total_compensation

In [ ]:
print(f"Estimated coefficient for feature '{input_data.columns[idx]}' is: {coefficient}")

In [ ]:
# Let's see how well we can estimate our target variable just by using `coefficients`.
# We will have to change the shape of coefficients such that it has the same number of rows
# as the input matrix has columns.
out_coefficients = np.zeros(shape=X_train.shape[1])
out_coefficients[idx] = coefficient
predictions = X_train @ out_coefficients

In [ ]:
print("Predicitons for total compensation:", predictions)

In [ ]:
# We can easily see that our predictions are significantly off from our target variable.
# So, we subtract current predictions from total compensation and we will use it in next
# iteration as the target variable. We call this variable residuals.
residuals = total_compensation - predictions
print("Residuals after 1st iteration:", residuals)

In [ ]:
# Let's create a block of code of what we have just seen implemented

# Standardize dataset
scaler = StandardScaler()
X_train = scaler.fit_transform(input_data)

# Define initial state of variables
residuals = total_compensation
indices = []
coefficients = np.zeros(shape=X_train.shape[1])

In [ ]:
def calculate_rmse(residuals):
    return np.sqrt(np.sum(residuals ** 2) / len(residuals))

In [ ]:
# 1st iteration
abs_contributions = np.abs(X_train.T @ residuals)
idx = np.argmax(abs_contributions)
indices.append(idx)
print(f"Selected index is: {idx}, i.e. '{input_data.columns[idx]}' feature.")

inverse_matrix = np.linalg.pinv(X_train[:, indices])
current_coefficients = inverse_matrix @ total_compensation
coefficients[indices] = current_coefficients
predictions = X_train @ coefficients
residuals = total_compensation - predictions
print("Residuals remaining to explain:", residuals)
print("RMSE:", calculate_rmse(residuals))

In [ ]:
# 2nd iteration
abs_contributions = np.abs(X_train.T @ residuals)
idx = np.argmax(abs_contributions)
indices.append(idx)
print(f"Selected index is: {idx}, i.e. '{input_data.columns[idx]}' feature.")

inverse_matrix = np.linalg.pinv(X_train[:, indices])
current_coefficients = inverse_matrix @ total_compensation
coefficients[indices] = current_coefficients
predictions = X_train @ coefficients
residuals = total_compensation - predictions
print("Residuals remaining to explain:", residuals)
print("RMSE:", calculate_rmse(residuals))

In [ ]:
# 3rd iteration
abs_contributions = np.abs(X_train.T @ residuals)
idx = np.argmax(abs_contributions)
indices.append(idx)
print(f"Selected index is: {idx}, i.e. '{input_data.columns[idx]}' feature.")

inverse_matrix = np.linalg.pinv(X_train[:, indices])
current_coefficients = inverse_matrix @ total_compensation
coefficients[indices] = current_coefficients
predictions = X_train @ coefficients
residuals = total_compensation - predictions
print("Residuals remaining to explain:", residuals)
print("RMSE:", calculate_rmse(residuals))

In [ ]:
# 4th iteration
abs_contributions = np.abs(X_train.T @ residuals)
idx = np.argmax(abs_contributions)
indices.append(idx)
print(f"Selected index is: {idx}, i.e. '{input_data.columns[idx]}' feature.")

inverse_matrix = np.linalg.pinv(X_train[:, indices])
current_coefficients = inverse_matrix @ total_compensation
coefficients[indices] = current_coefficients
predictions = X_train @ coefficients
residuals = total_compensation - predictions
print("Residuals remaining to explain:", residuals)
print("RMSE:", calculate_rmse(residuals))

In [ ]:
# We end the algorithm because we used all features. The residuals converge but 
# let's take a look at the error on the dataset.
rmse = calculate_rmse(residuals)
print("RMSE:", rmse)

In [ ]:
# The error above looks like we did not learn anything about total compensation from data.
# The missing term is intercept. Intercept is defined as `mean(y) - mean(x1)* coef1 - mean(x2) * coef2 - ...`
# where `y` is target `x1` is the 1st feature and `coef1` is the 1st coefficient
# corrresponding to the 1st feature. All terms except for `mean(y)` vanishes given that
# we standardized our input those features will have mean(x) = 0.
intercept = np.mean(total_compensation)
residuals = total_compensation - predictions - intercept

In [ ]:
# Let's delve into the error on the dataset once again.
rmse = np.sqrt(np.sum(residuals ** 2) / len(residuals))
print("RMSE:", rmse)

In [ ]:
# We can finally rewrite the steps into a function.
def omp(X, y, sparsity):
    num_atoms = X.shape[1]
    residuals = y
    selected_features = []
    coefficients = np.zeros(num_atoms)

    for _ in range(sparsity):
        abs_contributions = np.abs(X.T @ residuals)
        best_feature = np.argmax(abs_contributions)
        selected_features.append(best_feature)

        X_ = X[:, selected_features]
        curr_coefficients = np.linalg.pinv(X_) @ y
        coefficients[selected_features] = curr_coefficients

        prediction = X @ coefficients
        residuals = y - prediction

    intercept = y.mean()
    return coefficients, intercept

In [ ]:
#@title Solution
# Implementation of OMP algorithm
def omp(dictionary, signal, sparsity):
    num_atoms = dictionary.shape[1]
    residual = signal
    support = []
    coefficients = np.zeros(num_atoms)

    for _ in range(sparsity):
        correlations = np.abs(np.dot(dictionary.T, residual))
        best_atom = np.argmax(correlations)
        support.append(best_atom)

        selected_atoms = dictionary[:, support]
        projection = np.dot(np.linalg.pinv(selected_atoms), signal)
        coefficients[support] = projection

        prediction = np.dot(dictionary, coefficients)
        residual = signal - prediction

    intercept = signal.mean()
    return coefficients, intercept

### B. Sparse Signal Recovery: Use OMP to recover the original sparse signal from the measurements. Compare the obtained approximation with the ground truth signal.

In [ ]:
worker_data = pd.get_dummies(df, prefix_sep=" -> ", drop_first=False)
print(f"Our dataset has {worker_data.shape[0]} rows and {worker_data.shape[1]} columns.")

X_data = worker_data.drop("total_compensation", axis=1)
y_data = worker_data["total_compensation"]

X_train, X_test, y_train, y_test = train_test_split(
    X_data, y_data, test_size=0.20, random_state=12
)
column_names = X_train.columns

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

sparsity = 11
coefficients, intercept = omp(X_train, y_train, sparsity)

In [ ]:
# Calculate predictions using estimated coefficients and testing dataset
y_pred = # TODO
# Evaluate the model
rmse = metrics.mean_squared_error(y_test, y_pred, squared=False)
r2 = metrics.r2_score(y_test, y_pred)

print("Root Mean Squared Error:", rmse)
print("R-squared:", r2)

In [ ]:
#@title Solution
# Calculate predictions using estimated coefficients and testing dataset
y_pred = X_test @ coefficients + intercept
# Evaluate the model
rmse = metrics.mean_squared_error(y_test, y_pred, squared=False)
r2 = metrics.r2_score(y_test, y_pred)

print("Root Mean Squared Error:", rmse)
print("R-squared:", r2)

### C. Visualize Sparse Signal: Use coefficients estimated by OMP to visualize their impact using (e.g.: seaborn, matplotlib). Think about the interpretation of the visualization, especially "age", "job_level -> M8" and "generation -> Baby Boomers (1946 - 1964)".

In [ ]:
# Visualise the features that have been selected and their respective scores
feature_scores = pd.Series(coefficients, index=column_names).sort_values(ascending=False, key=lambda x: abs(x))
selected_features = feature_scores[:sparsity]

f, ax = plt.subplots(figsize=(10, 5))
shades = 31
palette = sns.color_palette("coolwarm", shades)
minmax = np.max([-np.min(selected_features), np.max(selected_features)])
bins = np.linspace(-minmax, minmax, num=shades)
palette_indices = np.digitize(selected_features, bins) - 1
colors = [palette[idx] for idx in palette_indices]

ax = sns.barplot(x=selected_features, y=selected_features.index, palette=colors, dodge=False)
ax.set_yticklabels(selected_features.index)
ax.set_xlabel("Feature impact on prediction")
plt.show()

In [ ]:
#@title Solution

# 1. "age"
# Since "age" is a continuous variable, the coefficient represents the difference in total compensation for each one-year difference in "age", if other variables remain the same.
# It means that if "age" of employees differs by one year (the rest is constant) total compensation will increase by 32k USD, on average.

# 2. "job_level -> M8" and "generation -> Baby Boomers (1946-1964)"
# Since both "job_level -> M8" and "generation -> Baby Boomers (1946-1964)" are categorical variables, the coefficient is the average difference in total compensation between
# the category for which, e.g. "job_level -> M8" = 0 (the reference group), i.e. employee is not at job level M8, and the category for which, e.g. "job_level -> M8" = 1 (the comparison group), i.e. employee is at job level M8.
# It means that if employee is at "job_level -> M8" (and other variables remain constant) total compensation will increase by 57k USD, on average.
# Next, it means that if employee was born as "generation -> Baby Boomers (1946-1964)" (and other variables remain constant) total compensation will decrease by 9k USD, on average.

# 4) Random Forest and Boruta feature selection coding example

In [ ]:
#Split the data to train and test set, with test size being 20%
X_data = worker_data.drop('total_compensation', axis=1)
y_data = worker_data['total_compensation']

X_train, X_test, y_train, y_test = train_test_split(
    X_data, y_data, test_size=0.20, random_state=12
)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

## Part A) Feature selection with Random Forest

### Step 1: Fit RF model without feature selection

In [ ]:
# Model specification
rf = RandomForestRegressor(n_estimators=150, max_depth=20, random_state=123)

# Fitting the model on the train set
rf.fit(X_train, y_train)

# Getting predictions on the test set
y_pred = rf.predict(X_test)

In [ ]:
# Prediction metrics without feature selection
print('Root Mean Squared Error (RMSE):', round(np.sqrt(metrics.mean_squared_error(y_test, y_pred)), 0))
mape = np.mean(np.abs((y_test - y_pred) / np.abs(y_test)))
print('Mean Absolute Percentage Error (MAPE):', round(mape * 100, 2), '%')

###Step 2: Select the most important features using variable importance scores

In [ ]:
# Specify number of features to select
MAX_FEATURES = 20

In [ ]:
# Fit the selection 
sel = SelectFromModel(RandomForestRegressor(n_estimators=150, max_depth=20, random_state=123), 
                      threshold=-np.inf,
                      max_features=MAX_FEATURES)
sel.fit(X_train, y_train)

In [ ]:
# Get the list of selected features - the ones with the highest feature importance
selected_feat = X_train.columns[(sel.get_support())]

In [ ]:
# Check how many and which features have been selected
n_feat = len(selected_feat)
print('Number of selected features: ', n_feat)

In [ ]:
# Visualise the features that have been selected and their respective scores
feature_scores = pd.Series(rf.feature_importances_, index=X_train.columns).sort_values(ascending=False)

f, ax = plt.subplots(figsize=(15, 5))
ax = sns.barplot(x=feature_scores[:n_feat], y=feature_scores[:n_feat].index, data=df)
ax.set_yticklabels(feature_scores[:n_feat].index, fontsize=7)
ax.set_xlabel("Feature importance score")
plt.show()

Note: By visualising importances of more variables, we would just by visual comparison come to a similar onclusion about a cut-off - the last variable to choose would be maybe P2 job level or length of service.

###Step 3: Fit the RF model again, now with selected features only

In [ ]:
X_train_sel = X_train[selected_feat]
X_test_sel = X_test[selected_feat]

In [ ]:
# Model specification
rf_sel = RandomForestRegressor(n_estimators=150, max_depth=20, random_state=123)

# Fitting the model on train data
rf_sel.fit(X_train_sel, y_train)

# Getting predictions on the test set
y_pred_sel = rf_sel.predict(X_test_sel)

In [ ]:
# Prediction metrics with feature selection
print('Root Mean Squared Error (RMSE):', round(np.sqrt(metrics.mean_squared_error(y_test, y_pred_sel)), 0))
mape = np.mean(np.abs((y_test - y_pred_sel) / np.abs(y_test)))
print('Mean Absolute Percentage Error (MAPE):', round(mape * 100, 2), '%')

*   At this point, we could use this model with selected features and fine-tune hyperparameters to obtain even better results

----

## Part B) Feature selection with Boruta

###Specify a base RF model and use it to run Boruta feature selection

In [ ]:
# Base RF model specification
rf = RandomForestRegressor(max_depth=20, random_state=123)

In [ ]:
# Boruta feature search
feat_selector = BorutaPy(
    verbose=2,
    estimator=rf,
    n_estimators=20,
    max_iter=25,
    random_state=123
)

feat_selector.fit(np.array(X_train), np.array(y_train))

In [ ]:
# Get the list of selected features
boruta_selected = []

for i in range(len(feat_selector.support_)):
    if (feat_selector.support_[i] or feat_selector.support_weak_[i]):
        boruta_selected.append(X_train.columns[i])
        print(X_train.columns[i])

In [ ]:
print('Number of selected features: ', len(boruta_selected))

##Exercise: Re-run Boruta selection with different parameters specification and use the selected features in random forest model.



### a) Run Boruta feature selection: use 50 trees in the forest and 20 iterations

In [ ]:
# Base RF model specification
rf = RandomForestRegressor(max_depth=20, random_state=123)

# Boruta feature search
feat_selector = BorutaPy(
    verbose=2,

    # ??
    
    random_state=123
)

feat_selector.fit(np.array(X_train), np.array(y_train))

In [ ]:
# Get the list of selected features
boruta_selected = []

for i in range(len(feat_selector.support_)):
    if (feat_selector.support_[i] or feat_selector.support_weak_[i]):
        boruta_selected.append(X_train.columns[i])
        print(X_train.columns[i])


In [ ]:
print('Number of selected features: ', len(boruta_selected))

###b) Run RF model with features selected by Boruta

In [ ]:
# Get subsets of explanatory variables only with features that were selected
X_filtered_train = #??
X_filtered_test = #??

In [ ]:
# Model specification 
rf = 

# Fit the model

# Get predictions
y_pred_BR_sel = 

In [ ]:
# Prediction metrics with feature selection
print('Root Mean Squared Error (RMSE):', np.sqrt(metrics.mean_squared_error(y_test, y_pred_BR_sel)))
mape = np.mean(np.abs((y_test - y_pred_BR_sel) / np.abs(y_test)))
print('Mean Absolute Percentage Error (MAPE):', round(mape * 100, 2))

In [ ]:
#@title Solution
rf = RandomForestRegressor(max_depth=20, random_state=123)

# Boruta feature search
feat_selector = BorutaPy(
    verbose=2,
    estimator=rf,
    n_estimators=50,
    max_iter=20,
    random_state=123
)

feat_selector.fit(np.array(X_train), np.array(y_train))

# Get the list of selected features
boruta_selected = []

for i in range(len(feat_selector.support_)):
    if (feat_selector.support_[i] or feat_selector.support_weak_[i]):
        boruta_selected.append(X_train.columns[i])
        print(X_train.columns[i])

# Get subsets of explanatory variables only with features that were selected
X_filtered_train = X_train[boruta_selected]
X_filtered_test = X_test[boruta_selected]

# Model specification
rf = RandomForestRegressor(n_estimators=150, max_depth=20, random_state=123)

# Fit the model
rf.fit(X_filtered_train, y_train)

# Get predictions
y_pred_BR_sel = rf.predict(X_filtered_test)

# Prediction metrics with feature selection
print('Root Mean Squared Error (RMSE):', np.sqrt(metrics.mean_squared_error(y_test, y_pred_BR_sel)))
mape = np.mean(np.abs((y_test - y_pred_BR_sel) / np.abs(y_test)))
print('Mean Absolute Percentage Error (MAPE):', round(mape * 100, 2))